# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [11]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

In [2]:
df = pd.read_csv('previsao_de_renda.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   index                  15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12466 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  mau                    15000 non-null  bool   
 15  re

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [4]:
# 1.
df = df.dropna()

X = df.drop(columns=['renda'])
y = df["renda"]

X = pd.get_dummies(X, drop_first=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=100)

In [5]:
# 2.
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
results_ridge = {}

for a in alphas:
    ridge = Ridge(alpha=a)
    ridge.fit(X_train, y_train)
    y_pred = ridge.predict(X_test)
    results_ridge[a] = r2_score(y_test, y_pred)

results_ridge

{0: 0.18574641159853778,
 0.001: 0.18574704837729783,
 0.005: 0.1857495783536407,
 0.01: 0.18575270280960143,
 0.05: 0.18577627794317242,
 0.1: 0.18580261967472733}

In [6]:
# 3.
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
results_lasso = {}

for a in alphas:
    lasso = Lasso(alpha=a, max_iter=10000)
    lasso.fit(X_train, y_train)
    y_pred = lasso.predict(X_test)
    results_lasso[a] = r2_score(y_test, y_pred)

results_lasso

C:\Users\henri\anaconda3\Lib\site-packages\sklearn\base.py:1389: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  return fit_method(estimator, *args, **kwargs)
C:\Users\henri\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
C:\Users\henri\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.610e+10, tolerance: 2.242e+07 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinat

{0: 0.18574641159921035,
 0.001: 0.18575091213893136,
 0.005: 0.1857687787928598,
 0.01: 0.18579080721811825,
 0.05: 0.18595249709870432,
 0.1: 0.18606211562858643}

In [7]:
# 4.
lr = LinearRegression()
sfs = SequentialFeatureSelector(lr, n_features_to_select="auto", direction="forward", cv=5)
sfs.fit(X_train, y_train)

selected_features = X_train.columns[sfs.get_support()]

lr.fit(X_train[selected_features], y_train)
y_pred = lr.predict(X_test[selected_features])
r2_stepwise = r2_score(y_test, y_pred)
r2_stepwise

0.18521286520520475

In [8]:
# 5.
pd.DataFrame({
    "Ridge": results_ridge,
    "Lasso": results_lasso,
    "Stepwise": r2_stepwise
})

,Ridge,Lasso,Stepwise
0.000,0.185746,0.185746,0.185213
0.001,0.185747,0.185751,0.185213
0.005,0.185750,0.185769,0.185213
0.010,0.185753,0.185791,0.185213
0.050,0.185776,0.185952,0.185213
0.100,0.185803,0.186062,0.185213


In [10]:
# 6.
X["idade2"] = X["idade"] ** 2
X["log_tempo_emprego"] = np.log1p(X["tempo_emprego"].fillna(0))

In [12]:
# 7.
tree = DecisionTreeRegressor(max_depth=5, random_state=42)
tree.fit(X_train, y_train)
y_pred = tree.predict(X_test)

r2_tree = r2_score(y_test, y_pred)
r2_tree

0.15076479446353974